In [1]:
import csv
import numpy as np
import pandas as pd
import time

In [2]:
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/MBiRW-master/Datasets/Datasets_indep/indep_asso.tsv")

In [45]:
print len(indep.drop_duplicates('drug'))
print len(indep.drop_duplicates('disease'))
indep.head(1)

115
313


,drug,disease,association
0,DB00035,D102100,0


In [3]:
indep_pos = indep[indep.association ==1]

In [34]:
sider_DrugBank_id = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3_3_SIDER_STITCHid_DrugBankid.tsv")
offside_DrugBank_id = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/4_3_OFFSIDE_STITCHid_DrugBankid.tsv")
sider_feature = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/3_4_SIDER_db_side_effect.tsv")
offside_feature = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/4_4_OFFSIDE_db_side_effect.tsv")

In [35]:
sider_found = sider_DrugBank_id[sider_DrugBank_id['DrugBank_ID'].isin(indep['drug'])]
print len(sider_found)
print len(sider_found.drop_duplicates(subset = 'DrugBank_ID'))
sider_found.head(1)

61
61


,STITCH_compound_id_flat,STITCH_compound_id_stereo,DrugBank_ID
9,CID100003075,CID000039186,DB00343


In [36]:
offside_found = offside_DrugBank_id[offside_DrugBank_id['DrugBank_ID'].isin(indep['drug'])]
print len(offside_found)
print len(offside_found.drop_duplicates(subset = 'DrugBank_ID'))
offside_found.head(1)

46
46


,STITCH_compound_id_stereo,DrugBank_ID
7,CID000000702,DB00898


In [37]:
sider_result = pd.DataFrame()
for index, row in sider_found.iterrows():
    li_row = row.tolist()
    flat = li_row[0]
    stereo = li_row[1]
    drugbank_id = li_row[2]
    filtered = sider_feature[(sider_feature.STITCH_compound_id_flat == flat) & (sider_feature.STITCH_compound_id_stereo == stereo)]
    for index2, row2 in filtered.iterrows():
        umls_id = row2['UMLS_concept_id_for_MedDRA']
        temp_df = pd.DataFrame([[drugbank_id, umls_id]], columns=['DrugBank_ID', 'umls_id'])
        sider_result = sider_result.append(temp_df)

In [38]:
print len(sider_result.drop_duplicates(subset = 'DrugBank_ID'))
print len(sider_result.drop_duplicates(subset = 'umls_id'))

61
1448


In [39]:
offside_result = pd.DataFrame()
for index, row in offside_found.iterrows():
    li_row = row.tolist()
    stereo = li_row[0]
    drugbank_id = li_row[1]
    filtered = offside_feature[offside_feature.stitch_id == stereo]
    for index2, row2 in filtered.iterrows():
        umls_id = row2['umls_id']
        temp_df = pd.DataFrame([[drugbank_id, umls_id]], columns=['DrugBank_ID', 'umls_id'])
        offside_result = offside_result.append(temp_df)

In [40]:
print len(offside_result.drop_duplicates(subset = 'DrugBank_ID'))
print len(offside_result.drop_duplicates(subset = 'umls_id'))

46
4471


In [41]:
frames = [sider_result, offside_result]
combined = pd.concat(frames).sort_values(by = 'DrugBank_ID').drop_duplicates()

In [42]:
print len(combined)
print len(combined.drop_duplicates(subset = 'DrugBank_ID')) # drug의 수

23173
77


In [44]:
filtered = combined[combined['umls_id'].isin(gold_se['umls_id'])]
print len(filtered)
print len(filtered.drop_duplicates(subset = 'DrugBank_ID')) # indep의 drug 수 
print len(filtered.drop_duplicates(subset = 'umls_id')) # 공통되는 feature의 수
filtered.head(1)

14816
77
1864


,DrugBank_ID,umls_id
0,DB00125,C0008301


In [48]:
indep_pos_se = indep_pos[indep_pos['drug'].isin(filtered['DrugBank_ID'])]
print len(indep_pos_se)
print len(indep_pos_se.drop_duplicates())
print len(indep_pos_se.drop_duplicates('drug'))
print len(indep_pos_se.drop_duplicates('disease'))
indep_pos_se.head(1)

98
98
77
39


,drug,disease,association
178,DB01200,D102300,1


In [49]:
omim = list()
for i in indep_pos_se['disease']:
    omim.append(int(i[1:]))
indep_pos_se['disease'] = omim
indep_pos_se.head(1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,drug,disease,association
178,DB01200,102300,1


In [52]:
gold_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/PREDICT_data/2_gold_asso.tsv")
indep_pos_se_dige = indep_pos_se[indep_pos_se['disease'].isin(gold_asso['OMIM_ID'])]
print len(indep_pos_se_dige)
print len(indep_pos_se_dige.drop_duplicates())
print len(indep_pos_se_dige.drop_duplicates('drug'))
print len(indep_pos_se_dige.drop_duplicates('disease'))
indep_pos_se_dige.head(2)

93
93
73
35


,drug,disease,association
178,DB01200,102300,1
601,DB00575,103780,1


In [54]:
indep_pos_se_dige.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/PREDICT_data/3_indep_asso.tsv", sep='\t', index=False)
indep_pos_se_dige.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/PREDICT_data/3_indep_asso.tsv", sep='\t', index=False)

In [53]:
temp = filtered[filtered['DrugBank_ID'].isin(indep_pos_se_dige['drug'])]
print len(temp)
print len(temp.drop_duplicates(subset = 'DrugBank_ID')) # indep의 drug 수 
print len(temp.drop_duplicates(subset = 'umls_id')) # 공통되는 feature의 수
temp.head(1)

14127
73
1844


,DrugBank_ID,umls_id
0,DB00125,C0008301


In [55]:
temp.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/3_indep_drug_se.tsv", sep='\t', index=False)
temp.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/3_indep_drug_se.tsv", sep='\t', index=False)

In [57]:
indep_gold_se = gold_se[gold_se['umls_id'].isin(temp['umls_id'])]
print len(indep_gold_se)
print len(indep_gold_se.drop_duplicates())
print len(indep_gold_se.drop_duplicates('DrugBank_ID'))
print len(indep_gold_se.drop_duplicates('umls_id'))

92321
92321
446
1844


In [58]:
indep_gold_se.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/4_gold_se_by_indep.tsv", sep='\t', index=False)
indep_gold_se.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/5_new_drug/PREDICT_data/4_gold_se_by_indep.tsv", sep='\t', index=False)